# Project II: Economic Growth 

This notebook will help you getting started with analyzing the growth dataset, `growth.csv`.

In [71]:
import pandas as pd 
import numpy as np 
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
import v2_tools as lm 
from sklearn.linear_model import Lasso

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read data 

In [41]:
dat = pd.read_csv('../data/growth.csv')
lbldf = pd.read_csv('../data/labels.csv', index_col='variable')
lbl_all = lbldf.label.to_dict() # as a dictionary
print(f'The data contains {dat.shape[0]} rows (countries) and {dat.shape[1]} columns (variables).')

The data contains 214 rows (countries) and 85 columns (variables).


# Collections of variables

In order to make the analysis simpler, it may be convenient to collect variables in sets that belong together naturally. 

In [42]:
# all available variables
vv_institutions = ['marketref', 'dem', 'demCGV', 'demBMR', 'demreg'] 
vv_geography = [
        'tropicar','distr', 'distcr', 'distc','suitavg','temp', 'suitgini', 'elevavg', 'elevstd',
        'kgatr', 'precip', 'area', 'abslat', 'cenlong', 'area_ar', 'rough','landlock', 
        'africa',  'asia', 'oceania', 'americas' # 'europe' is the reference
]
vv_geneticdiversity = ['pdiv', 'pdiv_aa', 'pdivhmi', 'pdivhmi_aa']
vv_historical = ['pd1000', 'pd1500', 'pop1000', 'pop1500', 'ln_yst'] # these are often missing: ['pd1', 'pop1']
vv_religion = ['pprotest', 'pcatholic', 'pmuslim']
vv_danger = ['yellow', 'malfal',  'uvdamage']
vv_resources = ['oilres', 'goldm', 'iron', 'silv', 'zinc']
vv_educ = ['ls_bl', 'lh_bl'] # secondary, tertiary: we exclude 'lp_bl' (primary) to avoid rank failure 

vv_all = {'institutions': vv_institutions, 
          'geography': vv_geography, 
          'geneticdiversity': vv_geneticdiversity,
          'historical': vv_historical,
          'religion': vv_religion,
          'danger':vv_danger, 
          'resources':vv_resources
         }
list_of_lists = vv_all.values()
vv_all['all'] = [v for sublist in list_of_lists for v in sublist]


In [43]:
# convenient to keep a column of ones in the dataset
dat['constant'] = np.ones((dat.shape[0],))

## Selection of variables

In [44]:
# remove rows with missing values of lgdp_initial and gdp_growth
df = dat.loc[(dat['lgdp_initial'].notna())&(dat['gdp_growth'].notna())]

# if no democracy data, set to 0
df.loc[df[['dem','demCGV','demBMR']].isna().any(axis=1), ['dem','demCGV','demBMR']] = 0

df

,code,marketref,dem,demCGV,demBMR,demreg,lp_bl,ls_bl,lh_bl,tropicar,...,capital_growth_pct_gdp_initial,capital_growth_pct_gdp_now,gdp_initial,gdp_now,investment_rate,gdp_growth,pop_growth,lgdp_initial,lpop_initial,constant
4,ARG,34.144062,0.0,0.0,0.0,0.071429,72.400000,15.300000,4.000000,0.027089,...,24.440095,13.776598,1.691742e+11,3.944470e+11,19.976606,0.004101,0.012921,8.865619,16.988575,1.0
7,AUS,29.444778,1.0,1.0,1.0,0.954545,29.300000,48.300000,21.500000,0.381887,...,33.022170,22.255543,3.266906e+11,1.446367e+12,27.672347,0.015481,0.014498,10.170480,16.341799,1.0
8,AUT,38.210518,1.0,1.0,1.0,0.954545,57.263283,35.800000,2.600000,0.000000,...,31.038141,25.193306,1.461597e+11,4.191863e+11,25.954993,0.017677,0.003556,9.881951,15.826015,1.0
10,BDI,NaN,0.0,0.0,0.0,0.032258,11.365340,0.912563,0.141201,1.000000,...,4.530795,11.377847,9.172368e+08,2.406362e+09,10.887358,-0.005276,0.024885,5.574601,15.062276,1.0
11,BEL,53.843560,1.0,1.0,1.0,0.954545,65.299510,28.700000,5.200000,0.000000,...,29.656689,24.764056,1.912545e+11,5.126385e+11,23.671420,0.016257,0.003600,9.893827,16.083043,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,VCT,54.545450,0.0,0.0,0.0,0.888889,NaN,NaN,NaN,NaN,...,NaN,NaN,1.654725e+08,7.382574e+08,NaN,0.026173,0.004082,7.511199,11.413116,1.0
185,ZAF,1.785714,0.0,0.0,0.0,0.032258,26.700000,27.400002,3.700000,0.037754,...,28.557312,12.426404,1.378055e+11,4.002289e+11,22.211417,0.001554,0.019968,8.739389,16.909720,1.0
187,ZMB,22.524733,0.0,0.0,0.0,0.032258,54.600000,1.900000,0.600000,1.000000,...,NaN,NaN,6.606381e+09,2.864814e+10,36.330155,-0.000287,0.030071,7.365705,15.245597,1.0
188,ZWE,0.000000,0.0,0.0,0.0,0.032258,51.920998,5.440012,2.309395,1.000000,...,18.337780,NaN,6.903321e+09,1.573755e+10,15.059719,-0.004174,0.020879,7.174070,15.481199,1.0


In [45]:
# get the maximum number of data
max_of_obs = df.shape[0]

var_has_no_obs = {}
for var in [col for col in df.columns if col not in ['code','lgdp_initial','gdp_growth']]:
    no_of_obs = df[var].notna().sum()
    var_has_no_obs[var] = no_of_obs

# sort the dictionary by the number of observations
sorted_var_has_no_obs = dict(sorted(var_has_no_obs.items(), key=lambda item: item[1], reverse=True))

# count how many variables have max_of_obs observations
count = 0
for key, value in sorted_var_has_no_obs.items():
    if value == max_of_obs:
        count += 1

print(f'There are {count} variables that have {max_of_obs} observations.')

There are 27 variables that have 102 observations.


In [49]:
keep_cols = [key for key, value in sorted_var_has_no_obs.items() if value ==max_of_obs]
all_cols = ['lgdp_initial','gdp_growth'] + keep_cols
df_master = df[all_cols]

# Double Post Lasso

In [89]:
y_labels = ['gdp_growth']
d_labels = ['lgdp_initial']
Z_labels = [col for col in df_master.columns if col not in y_labels+d_labels+['constant']]

# define the data
Z = df[Z_labels].values
d = df[d_labels].values
y = df[y_labels].values


# standardize the data
Z = (Z - np.mean(Z, axis=0)) / np.std(Z, axis=0, ddof=1)
d = (d - np.mean(d, axis=0)) / np.std(d, axis=0, ddof=1)
X = np.column_stack((d,Z))
# X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
# y = (y - np.mean(y, axis=0)) / np.std(y, axis=0)

# Define the number of samples and features
n = X.shape[0] # number of samples
p = df.shape[1] # number of features

In [90]:
# calculate penalty (BCCH)
x_chosen = X[:,:]
y_chosen = y[:]

lasso_ = lm.MyLasso_123(X=x_chosen, y=y_chosen)

print('Calculate BCCH \n','---'*10,)
print('Baby step')
penalty_term_obj = lm.penalty_term(X=x_chosen, y=y_chosen, alpha=0.05, c=1.1, n=n, p=p)
p_lambda = penalty_term_obj.bcch_pilot_rule()
beta_pilot = lasso_.lasso(p_lambda).coef_
print(f'We get a lambda={p_lambda:.8f}\n',beta_pilot,'\n', '---'*2)

# d
print('Grown up step')
residuals = y_chosen - x_chosen @ beta_pilot
bcch_lambda = penalty_term_obj.bcch_rule(residuals=residuals)

beta_bcch = lasso_.lasso(bcch_lambda).coef_
print(f'We get a lambda={bcch_lambda:.8f}\n', beta_bcch)
# print(f'lambda: {bcch_lambda:.6f} \n bcch betas\n', beta_bcch)

Calculate BCCH 
 ------------------------------
Baby step
We get a lambda=0.01082603
 [-0.  0.  0.  0. -0. -0.  0. -0.  0.  0. -0.  0.  0. -0. -0. -0. -0. -0.
  0.  0.  0. -0.  0.  0.  0. -0.  0.] 
 ------
Grown up step
We get a lambda=0.01593450
 [-0.  0.  0.  0. -0. -0.  0. -0.  0.  0. -0.  0.  0. -0. -0. -0. -0. -0.
  0.  0.  0. -0.  0.  0.  0. -0.  0.]


### Question 4.1
Estimate $\alpha$ using Douple Post Lasso (DPL).

Step 0: Calculate BRT

*Note:* In this exercise we will use the penalty suggested by BRT. BRT relies on homoscedasticity which is a strong assumption.

In [67]:
# Calculate BRT
penalty = bcch_lambda

    You should get: lambda_BRT = 3135.12

Step 1: Lasso Y using D and Z

*Hint:* To calculate the residuals from the LASSO-regression you can use the predict method from the Lasso object. The predict method returns the predicted values from the LASSO regression. You can then calculate the residuals by subtracting the predicted values from the actual values. 

In [110]:
# Run Lasso 
fit_BRTyx = Lasso(penalty, max_iter=10000).fit(X, y)
coefs=fit_BRTyx.coef_

y = y.reshape((n,))
d = d.reshape((n,))
# Calculate residuals
resyx = y-fit_BRTyx.predict(X)

# Calculate Y - Z@gamma (epsilon + alpha*d)
# Hint: You only need the variables given to you in this cell, in addition
# to a standardized data set you made previoously.
resyxz = resyx + d*coefs[0]

fit_BRTyx_foo = Lasso(penalty, max_iter=10000).fit(Z, y)

# Calculate residuals
resyxz_foo = y-fit_BRTyx_foo.predict(Z)

# Display first coefficient
print("First coefficient =",coefs[0])
print(resyx.shape)
print(resyxz.shape)
print(y.shape)
print(fit_BRTyx.predict(X).shape)

First coefficient = -0.0
(102,)
(102,)
(102,)
(102,)


In [ ]:
# # Setup data
# y = housing.median_house_value
# d = housing.median_income
# Z_basic = housing.drop(["median_house_value","median_income","ocean_proximity"],axis=1)

# # Add polynomial features
# # Hint: remember, you don't want the constant
# Z = PolynomialFeatures(3, include_bias=False).fit_transform(Z_basic)

# X = np.column_stack((d,Z))
# # Create a function for standardizing
# def standardize(X):

#     X_stan = (X - np.mean(X, axis=0))/np.std(X, axis=0, ddof=1)
#     return X_stan

# # Standardize data
# X_stan = standardize(X)
# Z_stan = standardize(Z)
# d_stan = standardize(d)

    You should get: First coefficient = 74248.24

Step 2: Lasso D using Z

In [111]:
# Run Lasso
fit_BRTdz = Lasso(penalty, max_iter=10000).fit(Z, d)
coefs=fit_BRTdz.coef_

# Calculate residuals
resdz=d-fit_BRTdz.predict(Z)
print(resdz.shape)

# Display first coefficient
print("First coefficient =",coefs[0].round(2))

(102,)
First coefficient = 0.0


    You should get: First coefficient = -0.55

Step 3: Estimate alpha

In [112]:
# Calculate alpha
num = resdz@resyxz
denom = resdz@d # WTF? hvorfor 
alpha_PDL = num/denom

# Display alpha
print("alpha_PDL = ",alpha_PDL.round(2))

alpha_PDL =  -0.0


    You should get: alpha_PDL =  40788.63

### Question 4.2
Calculate the implied variance estimate, $\check{\sigma}^2$, and calculate the standard deviation of $\check{\alpha}$.

In [113]:
# Calculate variance    
num = resdz**2@resyx**2/n
denom = (resdz.T@resdz/n)**2
sigma2_PDL = num/denom

# Display variance
print("sigma2_PDL = ",sigma2_PDL.round(2))

sigma2_PDL =  0.0


    You should get: sigma2_PDL =  4557181789.27

In [115]:
# Calculate standard error
se_PDL = np.sqrt(sigma2_PDL/n)

# Display standard error
print("se_PDL = ",se_PDL.round(2))

se_PDL =  0.0


    You should get: se_PDL =  472.26

### Question 4.3
Calculate the confidence interval for $\check{\alpha}$.

In [ ]:
# Calculate the quantile of the standard normal distribution that corresponds to the 95% confidence interval of a two-sided test
q = norm.ppf(1-0.025)

# Calculate confidence interval
CI_low_PDL  = alpha_PDL - q * se_PDL
CI_high_PDL = alpha_PDL + q * se_PDL

# Display confidence interval
print("CI_PDL = ",(CI_low_PDL.round(2),CI_high_PDL.round(2)))

CI_PDL =  (39863.01, 41714.24)
